In [1]:
from bs4 import BeautifulSoup as bs 
import requests as r
import pandas as pd

In [4]:
# Link from where you want to scrape the data from
Source_URL="https://www.amazon.com/s?k=xbox&crid=XQR3LJW42TZG&sprefix=xbox%2Caps%2C389&ref=nb_sb_noss_1"

# Custom HTTP header values as per the requirement
HEADERS={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36','Accept-Language': 'en-US, en;q=0.5'}

# Making HTTP request to the target page, Response 200 denotes that request was successful
r1=r.get(Source_URL,headers=HEADERS)
r1

<Response [200]>

In [10]:
a=r1.content

# Using BeautifulSoup to parse the fetched wbpae content in html format
webpage_r1=bs(a,'html.parser')

# Just to show the first few lines of fetched content
sample=str(webpage_r1)
print(sample[:1000])

<!DOCTYPE html>
<html class="a-no-js" data-19ax5a9jf="dingo" lang="en-us"><!-- sp:feature:head-start -->
<head><script>var aPageStart = (new Date()).getTime();</script><meta charset="utf-8"/>
<!-- sp:end-feature:head-start -->
<!-- sp:feature:csm:head-open-part1 -->
<script type="text/javascript">var ue_t0=ue_t0||+new Date();</script>
<!-- sp:end-feature:csm:head-open-part1 -->
<!-- sp:feature:cs-optimization -->
<meta content="on" http-equiv="x-dns-prefetch-control"/>
<link crossorigin="" href="https://images-na.ssl-images-amazon.com" rel="dns-prefetch"/>
<link crossorigin="" href="https://images-na.ssl-images-amazon.com" rel="preconnect"/>
<link crossorigin="" href="https://m.media-amazon.com" rel="dns-prefetch"/>
<link crossorigin="" href="https://m.media-amazon.com" rel="preconnect"/>
<link crossorigin="" href="https://completion.amazon.com" rel="dns-prefetch"/>
<link crossorigin="" href="https://completion.amazon.com" rel="preconnect"/>
<!-- sp:end-feature:cs-optimization -->
<!--

In [24]:
# Function for extracting the product title
def title_fetch(s):
    try:
        prod_name=s.find("span",attrs={'id':'productTitle'}).text.strip()
    except AttributeError:
        prod_name=""
    return prod_name

# Function for extracting the product price
def price_fetch(s):
    try:
        prod_price=s.find("span",attrs={'class':'a-offscreen'}).text.strip()
    except AttributeError:
        prod_price=""
    return prod_price

# Function for extracting the product rating
def rating_fetch(s):
    try:
        prod_rating=s.find("span",attrs={'class':'a-icon-alt'}).text.strip()
    except AttributeError:
        prod_rating=""
    return prod_rating

# Function for extracting the product reviews
def fetch_reviews(s):
    try:
        prod_rat=s.find("span",attrs={'id':'acrCustomerReviewText'}).text.strip()
    except AttributeError:
        prod_rat=""
    return prod_rat
    
# Function for extracting the product availability
def fetch_availability(s):
    try:
        prod_availability=s.find("span",attrs={'class':'a-size-medium a-color-success'}).text.strip()
    except AttributeError:
        prod_availability="Not Available"
    return prod_availability

In [25]:
# finding the links of all the products listed for our required product on the source web page
links=webpage_r1.find_all("a", attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})
link_fetch=[]
# storing all the links in a list
for i in links:
    temp=i.get('href')
    l='https://www.amazon.com/'+temp
    link_fetch.append(l)
dict={'Title':[],'Price':[],'Rating':[],'Number_of_Reviews':[],'Availability':[]}

for i in link_fetch:
    new_URL=i
    r2=r.get(new_URL,headers=HEADERS)
    b=r2.content
    webpage_r2=bs(b,'html.parser')
    dict['Title'].append(title_fetch(webpage_r2))
    dict['Price'].append(price_fetch(webpage_r2))
    dict['Rating'].append(rating_fetch(webpage_r2))
    dict['Number_of_Reviews'].append(fetch_reviews(webpage_r2))
    dict['Availability'].append(fetch_availability(webpage_r2))


    


In [26]:
print(dict)

{'Title': ['Upgraded RGB Cooling Fan Charging Station for Xbox Series S Console & Controller,Dual Charger Dock and Cooler System Stand with 15 Colorful Light Modes Accessories with 2*1400mAH Rechargeable Battery', 'Upgrade Cooling Fan &Dual Controller Charging Station for Xbox Series X with 15RGB Light, Vertical Stand Charger Dock with 2 Rechargeable Batteries Packs, Cooler System with Disc Accessories Storage', '$15 Xbox Gift Card [Digital Code]', 'Xbox Series X 1TB SSD Console - Includes Wireless Controller - Up to 120 frames per second - 16GB RAM 1TB SSD - Experience True 4K Gaming Velocity Architecture', 'Xbox Series S – Starter Bundle - Includes hundreds of games with Game Pass Ultimate 3 Month Membership - 512GB SSD All-Digital Gaming Console', '', 'Microsoft Xbox Series X Video Game 1TB Console + 1 Xbox Wireless Controller Black, 2160p Video Resolution, 4K Ultra HD Player, 120FPS, Wi-Fi, TSBEAU Silicone Controller Cover Skin + HDMI Cable', 'Xbox Series S Fortnite and Rocket Leag

In [29]:
df=pd.DataFrame(dict)
print(df.head())
df.to_csv("scraped_data.csv",header=True,index=False)

                                               Title    Price  \
0  Upgraded RGB Cooling Fan Charging Station for ...   $54.99   
1  Upgrade Cooling Fan &Dual Controller Charging ...   $57.99   
2                  $15 Xbox Gift Card [Digital Code]   $15.00   
3  Xbox Series X 1TB SSD Console - Includes Wirel...  $446.98   
4  Xbox Series S – Starter Bundle - Includes hund...  $289.00   

               Rating Number_of_Reviews   Availability  
0  4.6 out of 5 stars        81 ratings       In Stock  
1  4.1 out of 5 stars        95 ratings       In Stock  
2  4.6 out of 5 stars   178,307 ratings  Available now  
3  4.8 out of 5 stars    24,682 ratings       In Stock  
4  4.5 out of 5 stars       204 ratings       In Stock  
